In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override = True)

True

## Implementing a ChatGPT App with LangChain from Scratch

In [2]:
from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate


llm = ChatOpenAI(model_name='gpt-4o-mini', temperature=1)

prompt = ChatPromptTemplate(
    input_variables=["content"],
    messages=[
#         SystemMessage(content="You respond only in German."),
        SystemMessage(content='You are a chatbot having a conversation with a human.'),
        HumanMessagePromptTemplate.from_template("{content}")
    ]
)

chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True
)

while True:
    content = input('Your prompt: ')
    if content.lower() in ['quit', 'exit', 'bye']:
        print('Goodbye!')
        break
    
    response = chain.invoke({'content': content})
    print(response)
    print('-' * 50)

C:\Users\Akshat Shethia\AppData\Local\Temp\ipykernel_8276\803303118.py:21: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  chain = LLMChain(




> Entering new LLMChain chain...
Prompt after formatting:
System: You are a chatbot having a conversation with a human.
Human: 

> Finished chain.
{'content': '', 'text': 'Hello! How can I assist you today?'}
--------------------------------------------------


> Entering new LLMChain chain...
Prompt after formatting:
System: You are a chatbot having a conversation with a human.
Human: India is a ..

> Finished chain.
{'content': 'India is a ..', 'text': "India is a diverse and vibrant country located in South Asia. It is known for its rich history, cultural heritage, and a multitude of languages, religions, and traditions. With a population of over a billion people, India is the world's most populous democracy. The country has a mixed economy, characterized by agriculture, manufacturing, and services, and is one of the largest economies in the world. India is also famous for its contributions to art, music, dance, and cuisine, as well as its scenic landscapes ranging from the Himala

## Adding Chat Memory Using ConversationBufferMemory

In [5]:
from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory

llm = ChatOpenAI(model_name='gpt-4o-mini', temperature=1)

memory = ConversationBufferMemory(
    memory_key = "chat_history",
    return_messages = True
)


prompt = ChatPromptTemplate(
    input_variables=["content", "chat_history"],
    messages=[
#         SystemMessage(content="You respond only in German."),
        SystemMessage(content='You are a chatbot having a conversation with a human.'),
        MessagesPlaceholder(variable_name = "chat_history"), # Memory Will Be Stored Here
        HumanMessagePromptTemplate.from_template("{content}")
    ]
)

chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory = memory,
    verbose=True
)

while True:
    content = input('Your prompt: ')
    if content.lower() in ['quit', 'exit', 'bye']:
        print('Goodbye!')
        break
    
    response = chain.invoke({'content': content})
    print(response)
    print('-' * 50)



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a chatbot having a conversation with a human.
Human: Earth's Diameter is ?

> Finished chain.
{'content': "Earth's Diameter is ?", 'chat_history': [HumanMessage(content="Earth's Diameter is ?"), AIMessage(content="The average diameter of Earth is about 12,742 kilometers (7,918 miles). However, this can vary slightly depending on whether you're measuring the equatorial diameter or the polar diameter. The equatorial diameter is approximately 12,756 kilometers (7,926 miles), while the polar diameter is about 12,714 kilometers (7,900 miles), due to the Earth's slightly oblate shape.")], 'text': "The average diameter of Earth is about 12,742 kilometers (7,918 miles). However, this can vary slightly depending on whether you're measuring the equatorial diameter or the polar diameter. The equatorial diameter is approximately 12,756 kilometers (7,926 miles), while the polar diameter is about 12,714 kilometers (7,900 mil

## Saving Chat Sessions

In [10]:
from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage
from langchain.chains import LLMChain

# 1. Import FileChatMessageHistory
from langchain.memory import ConversationBufferMemory, FileChatMessageHistory

from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(model_name='gpt-4o-mini', temperature=1)

# 2. Add an additional keyword argument to the ConversationBufferMemory() constructor
history = FileChatMessageHistory('chat_history.json')
memory = ConversationBufferMemory(
    memory_key='chat_history',
    chat_memory=history,
    return_messages=True
)

prompt = ChatPromptTemplate(
    input_variables=["content", "chat_history"],
    messages=[
        SystemMessage(content="You are a chatbot having a conversation with a human."),
        MessagesPlaceholder(variable_name="chat_history"), 
        HumanMessagePromptTemplate.from_template("{content}")
    ]
)

chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory,
    verbose=False
)

while True:
    content = input('Your prompt: ')
    if content.lower() in ['quit', 'exit', 'bye']:
        print('Goodbye!')
        break
    
    response = chain.invoke({'content': content})
    print(response)
    print('-' * 50)

{'content': 'whats my name?', 'chat_history': [HumanMessage(content='Whats the speed of light?'), AIMessage(content="The speed of light in a vacuum is approximately 299,792 kilometers per second (or about 186,282 miles per second). In scientific notation, it's often rounded to \\(3.00 \\times 10^8\\) meters per second."), HumanMessage(content='whats the speed of light in vaccuum?'), AIMessage(content='The speed of light in a vacuum is approximately 299,792 kilometers per second (about 186,282 miles per second). This value is often represented in scientific notation as \\(3.00 \\times 10^8\\) meters per second.'), HumanMessage(content='whats its spped in a solid like ice?'), AIMessage(content="The speed of light in a solid, like ice, is slower than in a vacuum due to the medium's refractive index. For ice, the refractive index is about 1.31. To find the speed of light in ice, you can divide the speed of light in a vacuum by the refractive index:\n\n\\[\n\\text{Speed of light in ice} = \